### Setup
##### Imports

In [ ]:
import os, time, umap
import scipy.stats, scipy.signal
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib as mpl
import matplotlib.pyplot as plt

##### Entropy Utility Functions

In [ ]:
def PMF(waveform):                  # probability mass function of waveform
    hist = histogram(waveform)      # histogram of waveform
    return hist / sum(hist)         # normalize to sum of 1
    
def histogram(wave):
    "Returns a histogram of values with bin widths determined by the Freedman-Diaconis rule."
    n = wave.shape[0]                               # number of points in time series
    data_range = np.max(wave) - np.min(wave)        # range of voltage values
    bin_width = 2 * IQR(wave) / (n ** (1 / 3))      # width of each bin
    num_bins = math.ceil(data_range / bin_width)    # number of bins
    edges = bin_width*np.arange(num_bins+1)         # bin boundaries
    # return number of points in each bin
    return np.array([np.count_nonzero((edges[i]<=wave) & (wave<edges[i+1])) for i in range(num_bins)])

IQR = lambda X: np.quantile(X,.75) - np.quantile(X,.25)
outlier = lambda X: X>(np.quantile(X,.75) +IQR(X))

##### Hit Class

In [98]:
class Hit:
    "represents a waveform with a fixed number of voltage values over time"
    def __init__(self, test=-1, text_file=None, **kwargs):
        self.text_file = None
        self.waveform = None
        self.test = test
        self.time = None
        self.duration = None
        self.interval = None
        self.channel = None
        self.amplitude = None
        self.entropy = None
        self.spectrogram = None
        self.normalized = None

    def trimmed(self):                          # trim off trailing zeroes due to HLT
        return self.waveform[:self.get_duration()]
    
    def get_spectrogram(self):                  # calculate spectrogram of the waveform
        if self.spectrogram is None:
            self.spectrogram = scipy.signal.spectrogram(self.waveform, nperseg=120)
        return self.spectrogram
    
    def get_duration(self):                     # calculate number of points in the waveform, excluding the trailing HLT
        if self.duration is None:
            index = -1                      # start checking from end
            while self.waveform[index]==0:       # while no nonzero entries have been found
                index -= 1                  # iterate backwards through wave
            self.duration = index + len(self.waveform)
        return self.duration

    def get_entropy(self):                          # get information entropy value for the hit
        if self.entropy is None:
            dist = PMF(np.abs(self.trimmed()))      # probability distribution of voltage in wave
            self.entropy = scipy.stats.entropy(dist,base=2) # binary entropy of the PMF
        return self.entropy

    def ID(self):
        return self.text_file.split("\\")[-1].split(".")[0]

    def get_amplitude(self):                        # get amplitude value for the hit
        if self.amplitude is None:
            self.amplitude = 20*np.log10(np.max(np.abs(self.waveform))) + 80
        return self.amplitude
    
    def read_text(self, fname=None):
        fieldval = lambda id, s, val: s.split(": ")[-1] if id in s else val
        if fname is not None:
            self.text_file = fname
        with open(self.text_file, mode='r') as f:
            txt = f.readlines()
        for ind, line in enumerate(txt):            
            if line=="\n":
                break
            self.time = fieldval("TIME OF TEST:", line, self.time)
            self.interval = fieldval("SAMPLE INTERVAL (Seconds):", line, self.interval)
            self.channel = fieldval("CHANNEL NUMBER:", line, self.channel)
        else:
            raise ValueError(f"Expected blank line not found in waveform datafile {fname}")
        self.waveform = np.array(txt[ind+1:], dtype=float)

##### Dataset Class

In [99]:
class Dataset:
    def __init__(self, ID=-1):
        self.ID = ID                        # test number (or name) for dataset        
        self.hits = []                      # initial list of datapoints
        self.saveprops = ["waveform", "spectrogram", "entropy", "time",
                "amplitude", "channel", "text_file", "duration", "test"]
    
    def __getattr__(self, __name: str):     # gives easy access to hit attributes
        return np.array([hit.__getattribute__(__name) for hit in self.hits])

    def __len__(self) -> int:               # return number of hits from len() function
        return len(self.hits)

    def __getitem__(self,key):              
        if isinstance(key,int):             # for integer indices
            return self.hits[key]           # returns hit when dataset is indexed
        else:                               # for numpy-like boolean indexing
            new_hits = []
            for i in range(len(self.hits)):
                if key[i]:
                    new_hits.append(self.hits[i])
            new_set = Dataset()
            new_set.hits = new_hits     
            return new_set            

    def __add__(self,other):                # allows for datasets to be combined
        new = Dataset()                     # create new dataset
        new.hits = self.hits + other.hits   # set hits for new dataset
        return new                          # return combined dataset

    def read_txts(self, dir):               # read text files
        self.hits = []
        for file in os.listdir(dir):
            self.hits.append(Hit(test=self.ID))
            self.hits[-1].read_text(dir+"\\"+file)

    def set_property(self, key, array):
        for i, hit in enumerate(self.hits):
                hit.__setattr__(key,array[i])

    def normalize_data(self, offset):            
        x = np.log(self.spectrogram+offset)
        self.set_property("normalized", (x - np.min(x))/np.std(x))

    def save_npy(self, folder):
        dir = lambda s: folder + "\\" + s
        '''
        arrays = {"waves.npy": self.waveform,
                  "spectrograms.npy": self.spectrogram,
                  "entropy.npy": self.entropy,
                  "times.npy": self.time,
                  "amplitudes.npy": self.amplitude,
                  "channels.npy": self.channel,
                  "textfiles.npy": self.text_file,
                  "duration.npy":self.duration}'''
        for prop in self.saveprops:
            np.save(dir(prop+".npy"), self.__getattr__(prop))

    def load_npy(self, folder):
        dir = lambda s: folder + "\\" + s
        arrays = {}
        for f in os.listdir(folder):
            tag = f.split(".")[0]
            if tag in self.saveprops:
                arrays[tag] = np.load(dir(f), allow_pickle=True)
                n = arrays[tag].shape[0]
        self.hits = [Hit() for _ in range(n)]
        for tag, arr in arrays.items():
            self.set_property(tag,arr)

##### Load Data and normalize values

In [94]:
T = (Dataset(1), Dataset(2), Dataset(3), Dataset(4))
for i in range(1):
    T[i].read_txts(f"Waveform{i+1}")

data = T[0] + T[1] + T[2] + T[3]


##### Test/Train Split

In [ ]:
np.random.seed(42)
rng = np.random.rand(len(data))
train = rng<0.8

train_data = data[train]
test_data = data[~train]

In [ ]:
# Pixel intensity distribution
plt.figure(101, figsize=(7.5,5))
histvals = np.concatenate(np.concatenate(raw_specs))
h=plt.hist(histvals[np.isfinite(histvals)], bins=200, log=1)

plt.ylabel("Frequency in Dataset")
_ = plt.xlabel("Spectrogram Pixel Intensity")

# Pixel intensity distribution
plt.figure(102, figsize=(7.5,5))
histvals = np.concatenate(np.concatenate(specs))
h=plt.hist(histvals[np.isfinite(histvals)], bins=200, log=1)

plt.ylabel("Frequency in Dataset")
_ = plt.xlabel("Adjusted Spectrogram Pixel Intensity")

### Building an Autoencoder Model

In [ ]:
class Autoencoder(models.Model):
    def __init__(self):
        super(Autoencoder, self).__init__()   # initialize tensorflow model
        self.encoder = tf.keras.Sequential([  # convolutional model which reduces image down to 50 points
            layers.Input(shape=(61, 58, 1)),
            layers.Conv2D(32, 3, activation='relu'),
            layers.Conv2D(16, 3, activation='relu'),
            layers.Flatten(),
            layers.Dense(50)
            ])
        self.decoder = tf.keras.Sequential([  # inverse convolutional model which regenerates original image
            layers.Input(shape=(50)),
            layers.Dense(49248, activation='relu'),
            layers.Reshape((57, 54, 16)),
            #layers.Input(shape=(57, 54, 16)),
            layers.Conv2DTranspose(16, 3, activation='relu'),
            layers.Conv2DTranspose(16, 3, activation='relu'), 
            layers.Conv2DTranspose(8, 3, activation='relu'),
            layers.Conv2D(1, 3, activation='relu')
            ])

    def call(self, x):            # ensure model calls both encoder and decoder during training
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

checkpoint_path = "Model\\V04\\cp{epoch}.ckpt"

automodel = Autoencoder()
automodel.encoder.summary()
automodel.decoder.summary()
automodel.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError())

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                  save_weights_only=True,
                                                  verbose=1)

def encode(vals):
    '''Encode an spectrogram into the latent space and then decode it. 
        Completes task in small batches < 100 hits to avoid exhausting GPU memory.'''
    encoded_space = []
    decoded_space = []
    for i in range(0,len(vals), 100):
        j = min(len(vals), i+100)
        print(i, end=" ")
        #time.sleep(0.3)
        encoded_space.append(automodel.encoder(vals[i:j,:,:]))
        decoded_space.append(automodel.decoder(encoded_space[-1]))

    return np.concatenate(encoded_space), np.concatenate(decoded_space)

### Training the Autoencoder

In [ ]:
# weight_file = "cp9.ckpt"
weight_file = "v2cp10.ckpt"

if weight_file is None:         # train model and plot progress
    history = automodel.fit(x=train_data.normalized, y=train_data.normalized, epochs=20, 
                validation_data=(test_data.normalized, test_data.normalized), callbacks=cp_callback)
                
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label = 'val_loss')
    plt.ylabel("Loss")
    plt.xlabel("Epoch")
    plt.legend(); plt.show()
else:                           # load model from file
    automodel.load_weights(weight_file)

In [ ]:
latent, regenerated = encode(test_data.normalized)

In [ ]:
reducer = umap.UMAP()
flat = reducer.fit_transform(latent)

### Latent Space Plotting

In [100]:
def plot_latent(flat, color, label, savefile=None, show=True):
    zlim = [np.quantile(z,.995), np.quantile(z,.995)]
    cmap = mpl.cm.viridis
    f0 = [x for _, x in sorted(zip(s, flat[:,0]))]
    f1 = [x for _, x in sorted(zip(s, flat[:,1]))]
    z2 = [x for _, x in sorted(zip(s, z))]
    
    #plt.figure(100, figsize=(13,8))
    plt.scatter(f0, f1, s=7e4/len(f0), c=z2, cmap=cmap, vmin=zlim[0]+1, vmax=zlim[1]+2, alpha=1)
    plt.colorbar(label=label)
    if savefile is not None:
        plt.savefig(savefile, dpi=300)
    if show:
        plt.show()

In [ ]:
prefix = "Figures\\V04_"
save_figs = True
fig = lambda ID: f"{prefix}{ID}.png" if save_figs else None

plot_latent(flat, len_val/5e3, "Duration (ms)", fig(1))
plot_latent(flat, enta, "Entropy",fig(2))
plot_latent(flat, sns, "Channel #",fig(3))
plot_latent(flat, tst, "Specimen #",fig(4))
plot_latent(flat, amps[rng>0.8], "Amplitude (dB)",fig(5))

plot_latent(flat, enta, "Entropy", show=False)
xlim = plt.xlim(); ylim = plt.ylim()
box = [[-5,5],
       [ -6,  2.5]]
plt.plot(box[0]+box[0][::-1]+[box[0][0]], [box[1][0]]*2+[box[1][1]]*2+[box[1][0]])
plt.xlim(xlim); plt.ylim(ylim)
if save_figs:
       plt.savefig(fig(6), dpi=300)
plt.show()

### Plotting of High-Entropy Cluster

In [ ]:
cluster = np.array([box[0][0]<flat[i,0]<box[0][1] and box[1][0]<flat[i,1]<box[1][1] for i in range(flat.shape[0])])
reducer = umap.UMAP()
flat_cluster = reducer.fit_transform(latent[cluster,:])

In [ ]:
plot_latent(flat_cluster,np.array(enta)[cluster],"Entropy", fig(7))
plot_latent(flat_cluster,np.array(sns)[cluster],"Channel #", fig(8))
plot_latent(flat_cluster,np.array(amps)[rng>0.8][cluster],"Amplitude (dB)", fig(9))
plot_latent(flat_cluster,np.array(len_val/5e3)[cluster],"Duration (ms)", fig(10))

In [ ]:
mx = np.quantile(test_data, 0.999); print(mx)
for i in range(0, sp_val.shape[0], 201):
        fig, ax = plt.subplots(1,2, figsize=(10,10))
        print(i)
        ax[0].imshow(sp_val[i], origin="lower", vmin=0, vmax=mx)
        ax[1].imshow(b[i], origin="lower", vmin=0, vmax=mx)
plt.show()